## Fonction to Raster
Prend en entrée une Satellite Image, un dossier et un nom et la sauve en JP2 dans le dossier considéré

In [2]:
! pip install pyarrow -q -q -q 
! pip install rasterio -q -q -q 
! pip install geopandas -q -q -q
! pip install matplotlib -q -q -q
! pip install albumentations -q -q -q
! pip install pytorch_lightning -q -q -q
!pip install mlflow -q -q -q

In [52]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src')
from datetime import datetime
import numpy as np
import os

from utils.gestion_data import load_pleiade_data, write_splitted_images_masks, build_dataset_test, instantiate_module
from utils.utils import update_storage_access
from utils.labeler import RILLabeler
from datas.components.dataset import PleiadeDataset

import albumentations as album
from albumentations.pytorch.transforms import ToTensorV2
import torch

from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
)

from models.segmentation_module import SegmentationModule
from datas.datamodule import DataModule

import mlflow

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Définition fichier config
Ectrire ça en yaml une fois figé

In [ ]:

config = { 
    "tile size" : 250,
    "source train": "S2LOOKING",
    "type labeler" : "RIL", # None if source train != PLEIADE
    "buffer size" : 10, # None if BDTOPO
    "year"  : 2022,
    "territory" : "martinique",
    "dep" : "972",
    "n bands" : 3
}
config

In [53]:
config = { 
    "run name" : "entrainement martinique deeplab v3",
    "remote server uri":"https://projet-slums-detection-386760.user.lab.sspcloud.fr",
    "experiment name":"segmentation",
    "tile size" : 250,
    "source train": "PLEIADE",
    "type labeler" : "RIL", # None if source train != PLEIADE
    "buffer size" : 10, # None if BDTOPO
    "year"  : 2022,
    "territory" : "martinique",
    "dep" : "972",
    "n bands" : 3
}

config_train = { 
    "lr": 0.0001,
    "momentum": 0.9,
    "module" : "deeplabv3",
    "batch size" : 9,
    "max epochs" : 100
}

print(config)
print(config_train)

{'run name': 'entrainement martinique deeplab v3', 'remote server uri': 'https://projet-slums-detection-386760.user.lab.sspcloud.fr', 'experiment name': 'segmentation', 'tile size': 250, 'source train': 'PLEIADE', 'type labeler': 'RIL', 'buffer size': 10, 'year': 2022, 'territory': 'martinique', 'dep': '972', 'n bands': 3}
{'lr': 0.0001, 'momentum': 0.9, 'module': 'deeplabv3', 'batch size': 9}


In [54]:
# params 
tile_size = config["tile size"]
n_bands = config["n bands"]
dep = config["dep"]
territory = config["territory"]
year = config["year"]
buffer_size = config["buffer size"] 
source_train = config["source train"] 
type_labeler = config["type labeler"] 

module = config_train["module"]
batch_size = config_train["batch size"]

train_directory_name = "../splitted_data"

update_storage_access()
%env MLFLOW_S3_ENDPOINT_URL=https://minio.lab.sspcloud.fr
# DL des données du teritoire dont on se sert pour l'entraînement

if source_train == "PLEIADE":
    
    # Plus tard décliner avec change detection etc..
    dataset_train =  build_dataset_train(year,territory,type_labeler,train_directory_name,PleiadeDataset)
    load_pleiade_data(2020,"mayotte")
    dataset_test = build_dataset_test("../data/PLEIADES/2020/MAYOTTE/ORT_2020052526670967_0519_8586_U38S_8Bits.jp2",3,250,labeler)
    image_size = (tile_size,tile_size)
    
transforms_augmentation = album.Compose(
        [
            album.Resize(300, 300, always_apply=True),
            album.RandomResizedCrop(
                *image_size, scale=(0.7, 1.0), ratio=(0.7, 1)
            ),
            album.HorizontalFlip(),
            album.VerticalFlip(),
            album.Normalize(),
            ToTensorV2(),
       ]
    )

transforms_preprocessing = album.Compose(
        [
            album.Resize(*image_size, always_apply=True),
            album.Normalize(),
            ToTensorV2(),
        ]
)

## Instanciation modèle et paramètres d'entraînement
optimizer = torch.optim.SGD
optimizer_params = {"lr": config_train["lr"], "momentum":  config_train["momentum"]}
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau
scheduler_params = {}
scheduler_interval = "epoch"

model = instantiate_module(module)

data_module = DataModule(
    dataset= dataset_train,
    transforms_augmentation=transforms_augmentation,
    transforms_preprocessing=transforms_preprocessing,
    num_workers=1, 
    batch_size= batch_size,
    dataset_test = dataset_test
)


lightning_module = SegmentationModule(
    model=model,
    optimizer=optimizer,
    optimizer_params=optimizer_params,
    scheduler=scheduler,
    scheduler_params=scheduler_params,
    scheduler_interval=scheduler_interval,
)


checkpoint_callback = ModelCheckpoint(
    monitor="validation_IOU", save_top_k=1, save_last=True, mode="max"
)

early_stop_callback = EarlyStopping(
    monitor="validation_IOU", mode="min", patience=3
)

lr_monitor = LearningRateMonitor(logging_interval="step")

strategy ="auto"
list_callbacks = [lr_monitor, checkpoint_callback, early_stop_callback]

#!pip install mlflow
mlflow.end_run()

run_name = config["run name"]
remote_server_uri = config["remote server uri"]
experiment_name = config["experiment name"] 

mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_experiment(experiment_name)
mlflow.pytorch.autolog()

torch.cuda.empty_cache()
gc.collect()

with mlflow.start_run(run_name=run_name):

    trainer = pl.Trainer(
    callbacks= list_callbacks,
    max_epochs=config_train["max epochs"],
    num_sanity_val_steps=2,
    strategy=strategy,
    log_every_n_steps=2
    )
    
    trainer.fit(lightning_module, datamodule=data_module)
    
    print("test")
    trainer.test(lightning_module , datamodule= data_module)

env: MLFLOW_S3_ENDPOINT_URL=https://minio.lab.sspcloud.fr


2023/04/17 14:55:06 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.


## Test du modèle

In [ ]:
#trainer.checkpoint_callback.best_model_path
import mlflow
mlflow.end_run()

run_name = "modele deeplabV38"
remote_server_uri = "https://projet-slums-detection-386760.user.lab.sspcloud.fr"
experiment_name = "segmentation"

mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_experiment(experiment_name)
mlflow.pytorch.autolog()

output_ligthning = 'lightning_logs/'

with mlflow.start_run(run_id="0c7742ac29844f95ab49b05dc9c4842f"):
    model = DeepLabv3Module()
    
    lightning_module_checkpoint = lightning_module.load_from_checkpoint(
    checkpoint_path=output_ligthning+'version_12/checkpoints/epoch=34-step=24815.ckpt',
    model= model,
    optimizer=optimizer,
    optimizer_params=optimizer_params,
    scheduler=scheduler,
    scheduler_params=scheduler_params,
    scheduler_interval=scheduler_interval
                                         )
    #artifact_path = "modele_segmentation_deeplabv3"
    #mlflow.pytorch.log_model(lightning_module_checkpoint.model, artifact_path)

    trainer = pl.Trainer(
    callbacks= list_callbacks,
    max_epochs=50,
    num_sanity_val_steps=2,
    strategy=strategy,
    log_every_n_steps=2
    )
    trainer.test(lightning_module_checkpoint, datamodule=data_module)



- Ici j'évalue un batch à la mano à mettre dans evaluation model

In [ ]:

def evaluate_quick(datasetparam,model,idx):
    imagesat = SatelliteImage.from_raster(datasetparam.list_paths_images[idx],"976",n_bands = 3) 

    model.eval()
    model = model.to("cuda")
    #image, mask, dic = dataset_test[idx]
    image, mask, dic = datasetparam[idx]

    # mask réel
    plt.imshow(mask, cmap='binary', interpolation='none')
    plt.colorbar(ticks=[0, 1])
    plt.show()

    # image
    imagesat.plot([0,1,2])

    # model
    output = model(image.unsqueeze(0).to("cuda"))
    print(output.shape)
    mask_model = np.array(torch.argmax(output,axis= 1).squeeze(0).to("cpu"))
    print(np.sum(mask_model))
    # mask model
    plt.imshow(mask_model, cmap='binary', interpolation='none')
    plt.colorbar(ticks=[0, 1])
    plt.show()


In [55]:

evaluate_quick(dataset,model,70)
evaluate_quick(dataset_test,model,49)

NameError: name 'evaluate_quick' is not defined

- marrant de voir comment la normalisation detecte les logements..sérendipité
- qgis

## gestion MLFlow

In [36]:
from mlflow import MlflowClient
run = mlflow.get_run(run_id="0c7742ac29844f95ab49b05dc9c4842f")
#r.data.tags.items() if not k.startswith("mlflow.")}
artifacts = [f.path for f in MlflowClient().list_artifacts(run.info.run_id, "plots")]
#print_auto_logged_info()
artifacts

['plots/image_test.png']

In [37]:
#  MlflowClient().list_artifacts(run.info.run_id) ok pour lister les sous dossiers
MlflowClient().list_artifacts(run.info.run_id)
artifacts = [f.path for f in MlflowClient().list_artifacts(run.info.run_id, "plots")]
artifacts[0]

'plots/image_test.png'

In [38]:
print("run_id: {}".format(run.info.run_id))
print("params: {}".format(run.data.params))
print("metrics: {}".format(run.data.metrics))

path = mlflow.artifacts.download_artifacts(run_id = run.info.run_id)


run_id: 0c7742ac29844f95ab49b05dc9c4842f
params: {'monitor': 'validation_loss', 'mode': 'min', 'patience': '3', 'min_delta': '-0.0', 'stopped_epoch': '0', 'epochs': '100', 'optimizer_name': 'SGD', 'lr': '0.0001', 'momentum': '0.9', 'dampening': '0', 'weight_decay': '0', 'nesterov': 'False', 'maximize': 'False', 'foreach': 'None', 'differentiable': 'False'}
metrics: {'train_loss': 0.24770013988018036, 'train_loss_epoch': 0.24770013988018036, 'train_loss_step': 0.28203338384628296, 'validation_IOU': 0.49895596504211426, 'validation_loss': 0.2548619508743286, 'stopped_epoch': 34.0, 'restored_epoch': 31.0, 'best_score': 0.2544165551662445, 'wait_count': 3.0}


In [40]:
path # c'est le ^path où  sont téléchargés les artefacts en local

'/tmp/tmpf90n2ev1/'

In [ ]:
from PIL import Image
os.listdir(path+"plots/")
im = Image.open(path+"plots/image_test.png")
im.show()

**Save with pytorch**

TO DO :
- les images Guyane sont en 16 bits et MAyotte et martinique etc.. en 8 normal que ça deconne
- faire en sorte de calculer plusieurs grosse images sur le .test et trouver une autre image que mayotte pour le moment qui est en 255 couleurs... != du reste
- mettre une oimage de la validation en jeu
- mettre le calcul d'image dans la validation aussi ?
- mettre dans data set test une image du jeu de validatio à minima et une image de jeu externe etc..
- dégager aussi les RIL vides ? Oui
- labelstudio, segment anything model
- bien construire un beau jeu de test avec cet outil  label studio, labellisation à la main , pour evaluation pertinente !!
- mettre le [0,1,2] par défautdans les plots
- se servir du if batch idx == 10 recharger tout les fichiers des batchs précédents et en faire une satellite data, stocker également ce qui fera une seule image et donc une seule labelled satellite image ..
- dans le filename des images splittée changer avec les bonnes coordonnées dans lenom (trouvable dans la bounding box..)
- faire une fonction générer dataset test qui servira pour tous les tests
- test de lancement via invit de comande train
- généraliser la création de liste de file path selon le dataset souhaité
- créer un yaml de config et le logger dans mlflow
- contrîole sur le modèle qui est loggé dans ml flow ?
- resoudre le bug sur l'ambiguité du nombre d'elemeents dans le batch
- workshop fevrier

Reunion section
- 13 travaux
- Production de masques : Juin Juillet,
    - Esane normal + Branches
    - demographie  # RDV Nathanael + soutien code
    - maj EAP (Optionnel)
    - IPI ICA
    - Antipol, tab multi
    
rtauargus prez , veux-tu présenter ? nathanael ?
fideli ..
pour rtauargus mix des différentes présentations déjà faites :
que veux-on partager, pas NTTIS
différentes prez du worksho en Septembre dernier exemples bien senti

Présentation "paillettes" -> d'où vient la complexité des données ? décrire avec le graph
tau argus outil de référence, différnets types de complexité , une partie résolu liens tableaux, partie non résolue, partie analyse de la demande, + méta données non automatisées
-> rtauargus
-> 
-> création liste de tables c'est sympa ça demande une généralisation, une métadonnées nickel permettrait d'automatiser cette analyse
-> insee.fr exemple, overlezf, reprise présentation

- census bureau litterature reunion Julien

## S2Looking training !

dl

In [ ]:
update_storage_access()
environment = get_environment()
root_path = get_root_path()
bucket = environment["bucket"]
path_s3_s2looking = environment["sources"]["PAPERS"]["S2Looking"]
path_local_s2looking = environment["local-path"]["PAPERS"]

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

fs.download(
        rpath=f"{bucket}/{path_s3_s2looking}",
        lpath=f"../{path_local_s2looking}",
        recursive=True)

In [ ]:
import zipfile
with zipfile.ZipFile(f"../{path_local_s2looking}/S2Looking.zip", 'r') as zip_ref:
    zip_ref.extractall(f"../{path_local_s2looking}")

In [ ]:
#!pip install mlflow
import mlflow

run_name = "s2looking" # config
remote_server_uri = "https://projet-slums-detection-2439.user.lab.sspcloud.fr"
experiment_name = "segmentation" # config

mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_experiment(experiment_name)
# mlflow.pytorch.autolog() # logger la config


## Création Dataset
torch.cuda.empty_cache()
gc.collect()

## En faire une fonction ? : Idée faire une classe de préparation des données pour chaque data set chargé qui in fine sortirait une list path ?

### config, appeler une fionction de création de dataset ?
dir_data = "../data/paper_dataset/S2Looking/"
img1_train = [dir_data + "train/Image1/"+name for name in os.listdir(dir_data + "/train/Image1")]
img1_val = [dir_data + "train/Image1/"+name for name in os.listdir(dir_data + "/train/Image1")]

img1_train = [dir_data + "train/Image1/"+name for name in os.listdir(dir_data + "/train/Image1")]
img1_val = [dir_data + "val/Image1/"+name for name in os.listdir(dir_data + "/val/Image1")]
img1_test = [dir_data + "test/Image1/"+name for name in os.listdir(dir_data + "/test/Image1")]

img2_train = [dir_data + "train/Image2/"+name for name in os.listdir(dir_data + "/train/Image2")]
img2_val = [dir_data + "val/Image2/"+name for name in os.listdir(dir_data + "/val/Image2")]
img2_test = [dir_data + "test/Image2/"+name for name in os.listdir(dir_data + "/test/Image2")]

label_train = [dir_data + "train/label/"+name for name in os.listdir(dir_data + "/train/label")]
label_val = [dir_data + "val/label/"+name for name in os.listdir(dir_data + "/val/label")]
label_test = [dir_data + "test/label/"+name for name in os.listdir(dir_data + "/test/label")]

img1_path = np.concatenate([np.sort(img1_train),np.sort(img1_val),np.sort(img1_test)])
img2_path = np.concatenate([np.sort(img2_train),np.sort(img2_val),np.sort(img2_test)])
label_path = np.concatenate([np.sort(label_train),np.sort(label_val),np.sort(label_test)])

mono_image_dataset = ChangeDetectionS2LookingDataset(img1_path,img2_path,label_path)

# transforms incorporé dans les dataset a posteriori
image_size = (256,256) # cf la classe dataset
transforms_augmentation = album.Compose(
        [
            album.Resize(300, 300, always_apply=True),
            album.RandomResizedCrop(
                *image_size, scale=(0.7, 1.0), ratio=(0.7, 1)
            ),
            album.HorizontalFlip(),
            album.VerticalFlip(),
            album.Normalize(),
            ToTensorV2(),
       ]
    )

transforms_preprocessing = album.Compose(
        [
            album.Resize(*image_size, always_apply=True),
            album.Normalize(),
            ToTensorV2(),
        ]
)


## Instanciation modèle et paramètres d'entraînement

optimizer = torch.optim.SGD
optimizer_params = {"lr": 0.0001, "momentum": 0.9}
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau
scheduler_params = {}
scheduler_interval = "epoch"

model = DeepLabv3Module(nchannel = 6) # viens du data set qui concatene 2 images à 3 channels

##Instanciation des datamodule et plmodule

data_module = DataModule(
    mono_image_dataset= mono_image_dataset,
    transforms_augmentation=transforms_augmentation,
    transforms_preprocessing=transforms_preprocessing,
    num_workers=1, 
    batch_size=2
)


lightning_module = SegmentationModule(
    model=model,
    optimizer=optimizer,
    optimizer_params=optimizer_params,
    scheduler=scheduler,
    scheduler_params=scheduler_params,
    scheduler_interval=scheduler_interval,
)

checkpoint_callback = ModelCheckpoint(
    monitor="validation_loss", save_top_k=1, save_last=True, mode="min"
)

checkpoint_callback_2 = ModelCheckpoint(
    monitor="train_loss", save_top_k=1, save_last=True, mode="min"
)
early_stop_callback = EarlyStopping(
    monitor="validation_loss", mode="min", patience=3
)
lr_monitor = LearningRateMonitor(logging_interval="step")

strategy ="auto"

with mlflow.start_run(run_name=run_name):

    trainer = pl.Trainer(
    callbacks=[lr_monitor, checkpoint_callback,checkpoint_callback_2, early_stop_callback],
    max_epochs=100,
    num_sanity_val_steps=2,
    strategy=strategy,
    log_every_n_steps=2
    )
    trainer.fit(lightning_module, datamodule=data_module)
    
    lightning_module_checkpoint = lightning_module.load_from_checkpoint(
    checkpoint_path=trainer.checkpoint_callback.best_model_path
    model=model,
    optimizer=optimizer,
    optimizer_params=optimizer_params,
    scheduler=scheduler,
    scheduler_params=scheduler_params,
    scheduler_interval=scheduler_interval
                                     )
    artifact_path = "models/modele_change_detection_deeplabv3_on_s2_looking"
    mlflow.pytorch.log_model(lightning_module_checkpoint.model, artifact_path)
    
    